Using Qermit to solve an optimization problem is really simple. Let's solve a Constrained Quadratic Program of the form
$$
\max_{x, y} \left(-x^2 - y^2 + 1.5x + 2y\right); \ 0.1 \le 0.3x + 0.1y \le 2.7; \ x,y \in [0, 2]
$$
Implement this in Qermit like this:

In [2]:
import numpy as np
from qermit.problems import CQP

quadratic = -np.diag(np.ones(2))
linear = np.array([1.5, 2.0])
constant = 0

constraints_lower = np.array([0.1])
constraints_upper = np.array([2.7])
linear_constraints = np.array([[0.3, 0.1]])

lower_bounds = np.array([0.0, 0.0])
upper_bounds = np.array([2.0, 2.0])

cqp = CQP(quadratic, linear, constant,
    linear_constraints, constraints_lower, constraints_upper,
    lower_bounds, upper_bounds,
    var_type="continuous", maximize=True
)

Now, let's solve this problem on a classical QP solver like [OSQP](https://osqp.org/docs/interfaces/python.html#python-interface) and on a simulated Qiskit QAOA:

In [10]:
from qermit.backend import QiskitBackend
from qermit.optimizer.osqp import OSQPOptimizer
from qermit.optimizer.pycma import PyCMAOptimizer
from qermit.optimizer.qiskit import QAOA
from qermit.run.job import JobFactory

# Load a local quantum simulator
backend = QiskitBackend().get_local_simulator_backend()
qaoa_optimizer = QAOA(backend)

osqp_optimizer = OSQPOptimizer()
cma_optimizer = PyCMAOptimizer()

jobs = JobFactory.create_jobs_autoconvert(cqp, [osqp_optimizer, qaoa_optimizer, cma_optimizer])

for job in jobs:
    job_result = job.run()
    print(job_result)
    print("CQP value:", cqp.value(job_result.result))

JobResult(result=array([0.75, 1.  ]), optimizer_time=4.1e-05, conversion_time=0.0, problem_name='CQP', optimizer_name='OSQP', converter_name='')
CQP value: 1.5624999999999998
JobResult(result=array([0.83870968, 1.03225806]), optimizer_time=14.385658979415894, conversion_time=0.03815650939941406, problem_name='CQP', optimizer_name='Qiskit-QAOA', converter_name='CQP->QUBO')
CQP value: 1.5535900104058271
JobResult(result=array([0.75000001, 1.00000001]), optimizer_time=0.08965587615966797, conversion_time=0.0, problem_name='CQP', optimizer_name='PyCMA', converter_name='CQP->CP')
CQP value: 1.5624999999999998
